# Input Output (I/O) 

## Learning Objectives
- Understand the basic concepts of Input/Output (I/O) operations in Julia. 
- Perform file handling operations: opening, reading, writing, and closing files. 
- Potentially explore additional file formats such as CSV, Excel, and JSON 
- Use Julia packages for handling specific file formats efficiently
- Implement basic data processing tasks using various file formats

## Overview of I/O
**Input/Output (I/O)** operations involve reading data from external sources (input) and writing data out (output), often to files on disk. Working with files is essential for tasks like data processing, configuration, logging, etc. Julia provides built-in functions for basic text and binary file I/O and has packages for more complex formats (CSV, Excel, JSON, etc.)

In Julia, interacting with files typically uses the `open()` function along with `read` and `write` for data or higher-level convenience functions provided by packages. 

## File Handling Basics 
The typical steps for file I/O in any language are:
- **Open** the file (specifying a mode like read or write)
- **Read from or write to** the file.
- **Close** the file to ensure resources are freed, and data is flushed to disk.

In Julia, you can open a file using `open("filepath", "r")` for reading, `"w"` for writing (which truncates/creates the file), or `"a"` for appending to the end. The result of `open` is an IO stream object that you can use with `read`, `write`, etc. It's important to close the file when done (with `close(stream)`). Although Julia will close files when the object is garbage-collected, it's best to do it explicitly or use a do-block (shown later)

```Julia 
# Open a file for reading
file = open("data/input_data.txt", "r")
# ... (we could read from it here)
close(file)
```

This code opens the file `input_data.txt` in a folder `data` (relative path) and then immediately closes it. Currently, we don't do anything between opening and closing. 

## Reading from a text file 
To read text from a file, you have a few options: 
- `read(file, String)` will read all the contents of the file into a single string. 
- `readline(file)` will read one line (up to a newline character) as a string. 
- `readlines(file)` will return an array of strings, each being one line of the file. 
- You can also iterate through the lines in a file using `eachline(file)`, which is memory efficient for large files. 


## Exercise: Reading a Text File 
Create your own text file locally on your machine, and use the following code to read and print the content: 

```Julia
file = open("data/input_data.txt", "r")
content = read(file, String)   # read entire file into a string
close(file)
println("File contents:\n", content)
```

Once you have read the contents in, write it a new text file. 

## Reading and Writing Binary Files

Sometimes you need to work with raw bytes or store structured numeric data compactly. Julia treats binary I/O almost exactly like text I/O, but you read and write `UInt8` buffers or specific numeric types. 

### Writing a Binary File

To serialize raw bytes or typed numeric arrays into a compact, unformatted file, open a file in write mode and use `write(io, data)`. Below are two examples: one for writing a `Vector{UInt8}` of arbitrary bytes, and another for writing a `Vector{Int32}` in your machine’s native byte order.
- `write(io, bytes::Vector{UInt8})` emits each byte exactly as it appears in memory.
- `write(io, numbers::Vector{Int32})` emits each 32-bit integer in your machine’s native endianness.

In [12]:
# Prepare some sample data
bytes   = UInt8[0x01, 0xFF, 0x10, 0x20]    # raw bytes
numbers = Int32[1, -2, 3]                  # three 32-bit ints

# 1) Write raw bytes to raw.bin
open("data/raw.bin", "w") do io
    write(io, bytes)
end

# 2) Write the Int32 array to ints.bin (native endianness)
open("data/ints.bin", "w") do io
    write(io, numbers)
end

12

### Reading a binary file 

In the code below, the first approach reads the entire file into a `Vector{UInt8}` and then “views” that byte buffer as a different element type using `reinterpret`. This is a zero-copy operation (the same memory is reinterpreted).

In [13]:
# 1) Read all bytes from raw.bin as Vector{UInt8}
raw_bytes = open("data/raw.bin", "r") do io
    read(io)    # returns Vector{UInt8}
end

# 2) Read all bytes from ints.bin, then reinterpret them as Int32
ints_from_bytes = open("data/ints.bin", "r") do io
    raw = read(io)                  # raw :: Vector{UInt8}
    reinterpret(Int32, raw)         # Vector{Int32}, length = sizeof(raw) ÷ sizeof(Int32)
end

println("raw_bytes:       ", raw_bytes)
println("ints_from_bytes: ", ints_from_bytes)

raw_bytes:       UInt8[0x01, 0xff, 0x10, 0x20]
ints_from_bytes: Int32[1, -2, 3]


## Closing Files and Resources

Always remember to close files when done. Using the `do` block form of `open` is a safe pattern to ensure closure even if an error occurs during reading/writing. 

For example, you could write and read a text file with a `do` block as follows: 

```Julia 
# Writing to a file using the `do` block
open("hello.txt", "w") do file
    write(file, "Hello, Julia!")
end

# Reading from the file using the `do` block
open("hello.txt", "r") do file
    content = read(file, String)
    println("Content of hello.txt: ", content)
end
```
This approach ensures the file is properly closed even if something goes wrong while writing or reading. 

## Overview of Common File Formats

Julia supports a variety of file formats for data storage and manipulation. Below is an overview of some commonly used file formats along with links to detailed resources for each format.

### Delimited Files
Delimited text files (CSV, TSV, etc.) can be handled by Julia’s built-in `DelimitedFiles` standard library. It provides the `readdlm` and `writedlm` functions for reading and writing numeric and string arrays with customizable delimiters.

[Delimited Files in Julia](https://docs.julialang.org/en/v1/stdlib/DelimitedFiles/)

### CSV Files
CSV (Comma-Separated Values) files are one of the most widely used formats for storing tabular data. Julia provides excellent support for reading and writing CSV files using the `CSV.jl` package.

[CSV Files in Julia](https://csv.juliadata.org/stable/)

### JSON Files
JSON (JavaScript Object Notation) is a lightweight data interchange format. Julia can handle JSON files using the `JSON.jl` package.

[JSON Files in Julia](https://github.com/JuliaIO/JSON.jl)

### HDF5 Files
HDF5 (Hierarchical Data Format version 5) is designed to store large amounts of data. The `HDF5.jl` package provides support for reading and writing HDF5 files in Julia.

[HDF5 Files in Julia](https://github.com/JuliaIO/HDF5.jl)

### Parquet Files
Parquet is another columnar storage file format optimized for use with data analytics. The `Parquet.jl` package allows for reading and writing Parquet files in Julia.

[Parquet Files in Julia](https://github.com/JuliaIO/Parquet.jl)

### NetCDF Files
NetCDF (Network Common Data Form) is a set of software libraries and self-describing, machine-independent data formats. The `NCDatasets.jl` package is used for working with NetCDF files in Julia.

[NetCDF Files in Julia](https://github.com/Alexander-Barth/NCDatasets.jl)

### BSON Files
BSON (Binary JSON) is a binary representation of JSON-like documents. Julia supports BSON files using the `BSON.jl` package.

[BSON Files in Julia](https://github.com/JuliaIO/BSON.jl)

### JLD2 Files
[JLD2.jl](https://github.com/JuliaIO/JLD2.jl) provides a native-Julia serialization format (built on a subset of HDF5) for saving and loading arbitrary Julia data structures—without the need for the external HDF5 C library.


By using the above links, you can explore more about each file format and learn how to effectively use them in Julia.

## Exercise: Exploring Other File Formats

1. Select a file format from the list above (e.g., CSV, JSON, HDF5, NetCDF, etc.). Choose **one file format** that you believe will be most relevant to your work beyond this course.
2. Use the relevant Julia package to:
   - **Write** a small example dataset to a file in that format.
   - **Read** the file back into Julia and print the contents to verify the data roundtrip worked.
3. Use the `do` block form of `open` if applicable, to ensure the file is safely closed.

### Tips:
- If you're working with tabular data, try `CSV.jl`.
- For structured configuration or data exchange, `JSON.jl` might be a good fit.
- If you deal with scientific datasets, explore `NetCDF` or `HDF5`.

This is an opportunity to explore a file format that aligns with the kinds of data you typically work with, or hope to work with, in your own projects.

# End of Section Quiz

In [14]:
using JSON

function show_quiz_from_json(path)
    quiz_data = JSON.parsefile(path)

    html = """
    <style>
    .quiz-question {
        background-color: #6c63ff;
        color: white;
        padding: 12px;
        border-radius: 10px;
        font-weight: bold;
        font-size: 1.2em;
        margin-bottom: 10px;
    }

    .quiz-form {
        margin-bottom: 20px;
    }

    .quiz-answer {
        display: block;
        background-color: #f2f2f2;
        border: none;
        border-radius: 10px;
        padding: 10px;
        margin: 5px 0;
        font-size: 1em;
        cursor: pointer;
        text-align: left;
        transition: background-color 0.3s;
        width: 100%;
    }

    .quiz-answer:hover {
        background-color: #e0e0e0;
    }

    .correct {
        background-color: #4CAF50 !important;
        color: white !important;
        border: none;
    }

    .incorrect {
        background-color: #D32F2F !important;
        color: white !important;
        border: none;
    }

    .feedback {
        margin-top: 10px;
        font-weight: bold;
        font-size: 1em;
    }
    </style>

    <script>
    function handleAnswer(qid, aid, feedback, isCorrect) {
        // Reset all buttons for the question
        let buttons = document.querySelectorAll(".answer-" + qid);
        buttons.forEach(btn => {
            btn.classList.remove('correct', 'incorrect');
        });

        // Apply correct/incorrect to selected
        let selected = document.getElementById(aid);
        selected.classList.add(isCorrect ? 'correct' : 'incorrect');

        // Show feedback below the question
        let feedbackBox = document.getElementById('feedback_' + qid);
        feedbackBox.innerHTML = feedback;
        feedbackBox.style.color = isCorrect ? 'green' : 'red';
    }
    </script>
    """

    for (i, question) in enumerate(quiz_data)
        qid = "$i"
        html *= """<div class="quiz-question">$(question["question"])</div><form class="quiz-form">"""

        for (j, answer) in enumerate(question["answers"])
            aid = "q$(i)_a$(j)"
            feedback = answer["feedback"]
            correct = startswith(lowercase(feedback), "correct")
            html *= """
            <button type="button" class="quiz-answer answer-$qid" id="$aid"
                onclick="handleAnswer('$qid', '$aid', '$feedback', $(correct))">
                $(answer["answer"])
            </button>
            """
        end

        html *= """<div class="feedback" id="feedback_$qid"></div></form><hr>"""
    end

    display("text/html", html)
end


# Use the function
show_quiz_from_json("questions/summary_io.json")

Which of the following is the most memory-efficient way to read a very large text file line-by-line in Julia? 
 read(file, String)
 
 
 readline(file)
 
 
 eachline(file)
 
 
 readlines(file)
 
 In Julia, what is the recommended way to ensure a file is always closed after reading or writing, even if an error occurs during processing? 
 Manually call close(file) after reading/writing.
 
 
 Use try..catch..finally to close the file
 
 
 Use the do block form of open()
 
 
 Let Julias garbage collector handle it